In [1]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, optimizers
import numpy as np
import csv
import gc

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
# csvファイルを2次元配列lとして読み込み
with open('./data/pn_data.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    l = [row for row in reader]

# lの要素をstrからfloatにする
tmp1 = []
tmp2 = []
for i in l:
    tmp2 = [float(j) for j in i]
    tmp1.append(tmp2)
l = tmp1

del tmp1, tmp2
gc.collect()

22

以下、k分割交差検証をkerasでやるやり方コピペ

In [3]:
def build_model(*, i_shape, o_shape):
    model=Sequential()
    model.add(layers.Dense(1024, activation='relu', input_shape=(i_shape,)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(o_shape, activation='softmax'))
    #model.summary()
    sgd = optimizers.RMSprop(lr=0.001)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['acc'])

    return model

In [4]:
data = []
#for i in range(len(l)):
for i in range(30):        # テスト用！！！！！！！！！！！！！！！！！！！！！！！
    data.append(l[i])
del l
gc.collect()

length = []
for i in data:
    length.append(len(i))
max_len = max(length)
for i in data:
    while len(i) != max_len:
        i.append(0)
    i = np.array(i)
data = np.array(data)
del length, max_len
gc.collect()

0

In [5]:
with open('./data/score.txt', 'r', encoding='utf-8') as f:
    scores = f.read().split('\n')
scores = np.array(scores)
score = np.array(scores[scores!=''])
del scores
gc.collect()

# scoreの要素をstrからintにする
tmp = [int(i) for i in score]
score = tmp
del tmp
gc.collect()

# test用！！！！！！！！！！！！！！！！！！！！！！！！
score = score[0:30]

score = np.array(score)
score = score.reshape([len(score), 1])

NPデータで学習

In [6]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(data, score, test_size=0.3)
# test_sizeは教師データの割合
X_test = np.array(X_test)
Y_test = np.array(Y_test)

kf = KFold(n_splits=10, shuffle=True)    # n_splits分割交差検証
all_loss = []
all_val_loss = []
all_acc = []
all_val_acc = []
ep = 300    # 一回分の学習回数

for train_index, val_index in kf.split(X_train,Y_train):
    train_data = []
    train_label = []
    val_data = []
    val_label = []
    for t in train_index:
        train_data.append(X_train[t])
        train_label.append(Y_train[t])
    for v in val_index:
        val_data.append(X_train[v])
        val_label.append(Y_train[v])
    train_data  = np.array(train_data)
    train_label = np.array(train_label).reshape(len(train_label), 1)
    val_data   = np.array(val_data)
    val_label   = np.array(val_label).reshape(len(val_label), 1)

    model = build_model(i_shape=len(train_data[0]), o_shape=len(train_label[0]))
    history = model.fit(x = train_data,
                        y = train_label,
                        epochs = ep,
                        batch_size = 8,
                        verbose = 0,    # 0だとログが出ない, 1だと出る
                        validation_data = (val_data, val_label))

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    all_loss.append(loss)
    all_val_loss.append(val_loss)
    all_acc.append(acc)
    all_val_acc.append(val_acc)
    
    test_score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test loss:', test_score[0])
    print('Test accuracy:', test_score[1])
    
    print('\n')

del train_data, train_label, val_data, val_label
del model, history, test_score
del loss, val_loss, acc, val_acc
del train_index, val_index
del t, v
gc.collect()


ave_all_loss = np.array([np.mean([x[i] for x in all_loss]) for i in range(ep)])
ave_all_val_loss = np.array([np.mean([x[i] for x in all_val_loss]) for i in range(ep)])
ave_all_acc = np.array([np.mean([x[i] for x in all_acc]) for i in range(ep)])
ave_all_val_acc = np.array([np.mean([x[i] for x in all_val_acc]) for i in range(ep)])
del all_loss, all_val_loss, all_acc, all_val_acc
gc.collect()

print('loss = ', np.mean(ave_all_loss))
print('val_loss = ', np.mean(ave_all_val_loss))
print('acc = ', np.mean(ave_all_acc))
print('val_acc = ', np.mean(ave_all_val_acc))

np_result = {'loss':np.mean(ave_all_loss),
             'val_loss':np.mean(ave_all_val_loss),
             'acc':np.mean(ave_all_acc),
             'val_acc':np.mean(ave_all_val_acc)}
del ave_all_loss, ave_all_val_loss, ave_all_acc, ave_all_val_acc
gc.collect()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Test loss: -1029.35377870757
Test accuracy: 0.0057471264


Test loss: -1029.35377870757
Test accuracy: 0.0057471264


Test loss: -1029.35377870757
Test accuracy: 0.0057471264


loss =  -996.5782697503874
val_loss =  -996.559927554741
acc =  0.0
val_acc =  0.0


0

カタルシス1 `= ave_n * ave_p`

In [7]:
catharsis1 = []
fun_array = []
for d in data:
    fun = 0
    n = 0
    p = 0
    i = 0
    t = 0
    p_counter = 0
    while i < len(d):
        if d[i] < 0:
            t += 1
            n += d[i]
            fun_array.append(0)
            i += 1
        elif d[i] > 0:
            while i+p_counter < len(d) and d[i+p_counter] > 0:
                p += d[i+p_counter]
                p_counter += 1
            i += p_counter
            if t == 0:
                fun = 0
            else:
                fun = (n/t) * (p/p_counter)
            fun_array.append(fun)
            while p_counter > 1:
                fun_array.append(0)
                p_counter += -1
            n = 0
            p = 0
            t = 0
            p_counter = 0
        else:
            fun_array.append(0)
            i += 1
    catharsis1.append(fun_array)

del fun_array, d, fun, n, p, i, t, p_counter
gc.collect()





from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(catharsis1, score, test_size=0.2)
# test_sizeは教師データの割合
X_test = np.array(X_test)
Y_test = np.array(Y_test)

kf = KFold(n_splits=10, shuffle=True)    # n_splits分割交差検証
all_loss = []
all_val_loss = []
all_acc = []
all_val_acc = []
ep = 300    # 一回分の学習回数

for train_index, val_index in kf.split(X_train,Y_train):
    train_data = []
    train_label = []
    val_data = []
    val_label = []
    for t in train_index:
        train_data.append(X_train[t])
        train_label.append(Y_train[t])
    for v in val_index:
        val_data.append(X_train[v])
        val_label.append(Y_train[v])
    train_data  = np.array(train_data)
    train_label = np.array(train_label).reshape(len(train_label), 1)
    val_data   = np.array(val_data)
    val_label   = np.array(val_label).reshape(len(val_label), 1)

    model = build_model(i_shape=len(train_data[0]), o_shape=len(train_label[0]))
    history = model.fit(x = train_data,
                        y = train_label,
                        epochs = ep,
                        batch_size = 8,
                        verbose = 0,    # 0だとログが出ない, 1だと出る
                        validation_data = (val_data, val_label))

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    all_loss.append(loss)
    all_val_loss.append(val_loss)
    all_acc.append(acc)
    all_val_acc.append(val_acc)
    
    test_score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test loss:', test_score[0])
    print('Test accuracy:', test_score[1])
    
    print('\n')

del train_data, train_label, val_data, val_label
del model, history, test_score
del loss, val_loss, acc, val_acc
del train_index, val_index
del t, v
gc.collect()


ave_all_loss = np.array([np.mean([x[i] for x in all_loss]) for i in range(ep)])
ave_all_val_loss = np.array([np.mean([x[i] for x in all_val_loss]) for i in range(ep)])
ave_all_acc = np.array([np.mean([x[i] for x in all_acc]) for i in range(ep)])
ave_all_val_acc = np.array([np.mean([x[i] for x in all_val_acc]) for i in range(ep)])
del all_loss, all_val_loss, all_acc, all_val_acc
gc.collect()

print('loss = ', np.mean(ave_all_loss))
print('val_loss = ', np.mean(ave_all_val_loss))
print('acc = ', np.mean(ave_all_acc))
print('val_acc = ', np.mean(ave_all_val_acc))

kata1 = {'loss':np.mean(ave_all_loss),
         'val_loss':np.mean(ave_all_val_loss),
         'acc':np.mean(ave_all_acc),
         'val_acc':np.mean(ave_all_val_acc)}
del ave_all_loss, ave_all_val_loss, ave_all_acc, ave_all_val_acc
gc.collect()

ResourceExhaustedError: OOM when allocating tensor with shape[434481,1024] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node RMSprop_3/RMSprop/update_dense_12/kernel/Square}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


カタルシス2 = ave_n + ave_p

In [ ]:
catharsis1 = []
fun_array = []
for d in data:
    fun = 0
    n = 0
    p = 0
    i = 0
    t = 0
    p_counter = 0
    while i < len(d):
        if d[i] < 0:
            t += 1
            n += d[i]
            fun_array.append(0)
            i += 1
        elif d[i] > 0:
            while i+p_counter < len(d) and d[i+p_counter] > 0:
                p += d[i+p_counter]
                p_counter += 1
            i += p_counter
            if t == 0:
                fun = 0
            else:
                fun = (n/t) + (p/p_counter)
            fun_array.append(fun)
            while p_counter > 1:
                fun_array.append(0)
                p_counter += -1
            n = 0
            p = 0
            t = 0
            p_counter = 0
        else:
            fun_array.append(0)
            i += 1
    catharsis1.append(fun_array)

del fun_array, d, fun, n, p, i, t, p_counter
gc.collect()





from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(catharsis1, score, test_size=0.2)
# test_sizeは教師データの割合
X_test = np.array(X_test)
Y_test = np.array(Y_test)

kf = KFold(n_splits=10, shuffle=True)    # n_splits分割交差検証
all_loss = []
all_val_loss = []
all_acc = []
all_val_acc = []
ep = 300    # 一回分の学習回数

for train_index, val_index in kf.split(X_train,Y_train):
    train_data = []
    train_label = []
    val_data = []
    val_label = []
    for t in train_index:
        train_data.append(X_train[t])
        train_label.append(Y_train[t])
    for v in val_index:
        val_data.append(X_train[v])
        val_label.append(Y_train[v])
    train_data  = np.array(train_data)
    train_label = np.array(train_label).reshape(len(train_label), 1)
    val_data   = np.array(val_data)
    val_label   = np.array(val_label).reshape(len(val_label), 1)

    model = build_model(i_shape=len(train_data[0]), o_shape=len(train_label[0]))
    history = model.fit(x = train_data,
                        y = train_label,
                        epochs = ep,
                        batch_size = 8,
                        verbose = 0,    # 0だとログが出ない, 1だと出る
                        validation_data = (val_data, val_label))

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    all_loss.append(loss)
    all_val_loss.append(val_loss)
    all_acc.append(acc)
    all_val_acc.append(val_acc)
    
    test_score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test loss:', test_score[0])
    print('Test accuracy:', test_score[1])
    
    print('\n')

del train_data, train_label, val_data, val_label
del model, history, test_score
del loss, val_loss, acc, val_acc
del train_index, val_index
del t, v
gc.collect()


ave_all_loss = np.array([np.mean([x[i] for x in all_loss]) for i in range(ep)])
ave_all_val_loss = np.array([np.mean([x[i] for x in all_val_loss]) for i in range(ep)])
ave_all_acc = np.array([np.mean([x[i] for x in all_acc]) for i in range(ep)])
ave_all_val_acc = np.array([np.mean([x[i] for x in all_val_acc]) for i in range(ep)])
del all_loss, all_val_loss, all_acc, all_val_acc
gc.collect()

print('loss = ', np.mean(ave_all_loss))
print('val_loss = ', np.mean(ave_all_val_loss))
print('acc = ', np.mean(ave_all_acc))
print('val_acc = ', np.mean(ave_all_val_acc))

kata2 = {'loss':np.mean(ave_all_loss),
         'val_loss':np.mean(ave_all_val_loss),
         'acc':np.mean(ave_all_acc),
         'val_acc':np.mean(ave_all_val_acc)}
del ave_all_loss, ave_all_val_loss, ave_all_acc, ave_all_val_acc
gc.collect()

カタルシス3 = sum_n * sum_p

カタルシス4 = sum_n + sum_p

カタルシス5 `= t * n + p`

カタルシス6 `= b * (t * n + p)`

In [ ]:
#np_ersult
#kata1
#kata2
#kata3
#kata4
#kata5
#kata6